# Desenvolvimento de um Modelo de Classificação de Dígitos MNIST usando TensorFlow

Este notebook descreve passo a passo a criação e o treinamento de uma Rede Neural Convolucional (CNN) para classificar imagens do dataset **MNIST**. O modelo é implementado utilizando **TensorFlow/Keras**.

---

## **1. Importação das Bibliotecas**

Primeiro, importamos as bibliotecas necessárias:
- `pathlib` para manipulação de caminhos ao salvar o modelo.
- `tensorflow` para implementação da rede neural e treinamento.
- `mnist` para carregar o conjunto de dados MNIST.

In [ ]:
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D

: 

## 2. Carregamento do Dataset MNIST
O MNIST é um conjunto de dados que contém imagens de dígitos escritos à mão, com resolução de 28x28 pixels em escala de cinza.

Carregamos os dados e dividimos em:

- `x_train` e `y_train`: dados de treinamento (imagens e rótulos).
- `x_test` e `y_test`: dados de teste (imagens e rótulos).

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

## 3. Pré-processamento dos Dados
### 3.1 Normalização

Normalizamos os valores dos pixels das imagens para o intervalo [0, 1], pois isso acelera o treinamento do modelo e melhora a performance.

In [ ]:
x_train = x_train / 255.0
x_test = x_test / 255.0

# 3.2 Redimensionamento
Redimensionamos as imagens para adicionar uma dimensão de canal, uma vez que as redes convolucionais esperam dados no formato (altura, largura, canais).

x_train e x_test passam de (N, 28, 28) para (N, 28, 28, 1), onde 1 representa o canal (escala de cinza).

In [ ]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

## **4. Construção do Modelo CNN**

Neste tópico, criamos a **Rede Neural Convolucional (CNN)**, uma arquitetura projetada para trabalhar com dados espaciais, como imagens. As CNNs utilizam camadas especiais para extrair características importantes das imagens, como bordas, texturas e padrões, que são essenciais para tarefas de classificação.

### **Estrutura da Arquitetura**

A CNN é composta por várias camadas que desempenham papéis distintos no processo de aprendizado. Abaixo estão as descrições detalhadas das camadas utilizadas:

1. **Camadas Convolucionais (`Conv2D`)**
   - As camadas convolucionais aplicam filtros (kernels) às imagens de entrada para extrair **características espaciais** como bordas, texturas e padrões.
   - Cada filtro realiza uma operação de convolução sobre a imagem, produzindo um **mapa de características**.
   - A quantidade de filtros determina quantos mapas de características serão produzidos.
   - **Ativação ReLU** (Rectified Linear Unit) é utilizada para introduzir **não-linearidade** no modelo, removendo valores negativos e mantendo os positivos.

   - **Exemplo**:  
     ```python
     Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1))
     ```
     - **32**: Número de filtros (ou mapas de características).
     - **(3, 3)**: Tamanho do kernel (filtro).
     - **activation='relu'**: Função de ativação ReLU.
     - **input_shape=(28, 28, 1)**: Dimensão da imagem de entrada (altura, largura, número de canais).

2. **Camadas de Pooling (`MaxPooling2D`)**
   - As camadas de pooling são responsáveis por **reduzir a dimensionalidade** dos mapas de características gerados pelas camadas convolucionais.
   - O **MaxPooling** seleciona o maior valor dentro de uma janela específica (geralmente 2x2), reduzindo a resolução da imagem e mantendo as informações mais importantes.
   - Isso ajuda a:
     - Reduzir o número de parâmetros e o custo computacional.
     - Controlar o **overfitting**.
     - Tornar o modelo mais robusto a pequenas variações nas imagens.

   - **Exemplo**:
     ```python
     MaxPooling2D((2, 2))
     ```
     - **(2, 2)**: Tamanho da janela de pooling.

3. **Camada Flatten**
   - A camada `Flatten` transforma a **matriz 2D** dos mapas de características em um **vetor 1D**.  
   - Essa etapa é necessária porque as camadas densas (Fully Connected) esperam entradas unidimensionais.  

   - **Exemplo**:  
     ```python
     Flatten()
     ```
     - Nenhum hiperparâmetro adicional é necessário.

4. **Camada Densa (`Dense`)**
   - Camadas totalmente conectadas (Fully Connected Layers) processam os vetores extraídos pelas camadas convolucionais e pooling.
   - A camada `Dense` conecta todos os neurônios de entrada a todos os neurônios de saída, permitindo ao modelo aprender combinações das características extraídas.
   - Utilizamos:
     - **Camada intermediária** com 128 neurônios e ativação **ReLU**: para modelar relações não-lineares.
     - **Camada de saída** com 10 neurônios e ativação **softmax**: para classificar as imagens em 10 classes (dígitos de 0 a 9). A função softmax converte os valores em **probabilidades** para cada classe.

   - **Exemplo**:
     ```python
     Dense(128, activation='relu')
     Dense(10, activation='softmax')
     ```
     - **128**: Quantidade de neurônios na camada intermediária.
     - **10**: Quantidade de neurônios na camada de saída (uma para cada classe).

---

### **Resumo da Arquitetura**

- **Camada 1**:  
   - `Conv2D`: 32 filtros, tamanho `(3x3)`, ativação ReLU.  
   - `MaxPooling2D`: Reduz as dimensões com uma janela `(2x2)`.

- **Camada 2**:  
   - `Conv2D`: 64 filtros, tamanho `(3x3)`, ativação ReLU.  
   - `MaxPooling2D`: Reduz as dimensões com uma janela `(2x2)`.

- **Camada 3**:  
   - `Flatten`: Achata os mapas de características em um vetor unidimensional.

- **Camada 4**:  
   - `Dense`: 128 neurônios, ativação ReLU.

- **Camada 5**:  
   - `Dense`: 10 neurônios, ativação softmax (saída com probabilidades para cada classe).


In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

## **5. Compilação do Modelo**

Compilamos o modelo com os seguintes parâmetros:
- **Optimizer**: Adam - um otimizador eficiente para aprendizado.
- **Loss**: Sparse Categorical Crossentropy - usado para classificação multiclasse, especialmente quando os rótulos estão no formato inteiro (não one-hot encoded).
- **Métricas**: Accuracy - métrica principal para avaliar o desempenho do modelo durante o treinamento e validação.


In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

## **6. Treinamento do Modelo**

Treinamos o modelo usando os dados de treinamento com as seguintes configurações:
- **Epochs**: 5 - o modelo passa por todo o conjunto de dados de treinamento 5 vezes.
- **Batch size**: 32 - os dados são processados em lotes de 32 amostras para melhor uso da memória.
- **Validation split**: 10% dos dados de treinamento são usados para validação, permitindo monitorar o desempenho do modelo em dados não vistos durante o treinamento.


In [ ]:
model.fit(x_train, y_train, epochs=5, batch_size=32, validation_split=0.1)

## **7. Salvando o Modelo Treinado**

Após o treinamento, salvamos o modelo em um arquivo no formato **HDF5** com a extensão `.h5`. Esse formato armazena a arquitetura, os pesos e o estado do otimizador do modelo, permitindo reutilizá-lo posteriormente para inferência ou continuação do treinamento.

Utilizamos a biblioteca `pathlib` para construir o caminho de salvamento do arquivo.


In [ ]:
path = Path('model.h5')
model.save(path)